In [1]:
import torch

In [2]:
the_model = torch.load('../CBOW_(F)_FullyTrained.obj')
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

pos_bucket = ['excited', 'happy', 'strength', 'joy', 'inspiration'] #happy, strength
neg_bucket = ['depressed','upset', 'hate', 'tired', 'warning', 'shock']
neut_bucket = ['neutral', 'formal', 'professional', 'passive', 'factual']

sentences =[('he lived a great life', pos_bucket),
            ('I\'m so sad', neg_bucket),
            ('living life is a pleasure not known to many', pos_bucket),
            ('I hate this', neg_bucket),
            ('parents take care of children', neut_bucket),
            ('the interview was neither funny nor witty', neg_bucket),
            ('you are such a liar', neg_bucket),
            ('youth is a celebration It\'s about our freedom And within that freedom is our joy', pos_bucket),
            ('there\'s power in mental healing', pos_bucket),
            ('wasting time is the mother of all problems', neg_bucket),
            ('I don\'t want to talk to you ever again', neg_bucket),
            ('behind every successful man there is a successful woman', pos_bucket),
            ('you should not count your chickens until they hatch', neut_bucket),
            ('you cause problems wherever you go', neg_bucket),
            ('you should not be joking around the office', neg_bucket),
            ('we should come together in times of distress', pos_bucket),
            ('where there is a will there is a way', pos_bucket),
            ('the enemy of the state is the enemy of the country', neg_bucket),
            ('hurricanes are dangerous to every form of life', neut_bucket),
            ('peaceful protests are acceptable', neut_bucket),
            ('center your mind and pursue your goals', pos_bucket),
            ('Be a society activist for truth', pos_bucket),
            ('the genius of a few changes the lives of many', pos_bucket),
            ('the powerful take control of all the wealth', neg_bucket),
            ('There are seven days in a week', neut_bucket),
            ('India is a country in Asia', neut_bucket),
            ('The olympic games are held every 4 years', neut_bucket),
            ('The average lifespan is 100 years', neut_bucket),
            ('John was found murdered in his house yesterday', neut_bucket),
            ('The graceful steps of the gymnast won him a gold medal', pos_bucket),
            ('People who dislike everyone and everything are just stupid', neg_bucket),
            ('The prince got married and they lived happily ever after', pos_bucket),
            ('Build your experience slowly and one day you will be successful', pos_bucket),
            ('Sadly his visa expired and he was deported from the country', neg_bucket),
            ('He cried for the whole month when he learnt about his family', neg_bucket),
            ('Your dirty mouth will not be tolerated', neg_bucket),
            ('Stop talking when I ask you to', neg_bucket),
            ('The unfortunate world we live in today', neg_bucket),
            ('His lectures are horrible even with good presentation slides', neg_bucket),
            ('He made it in his life after years of struggle', pos_bucket),
            ('Computers are a radical instrument of change', pos_bucket),
            ('Chemistry is the study of molecular reactions', neut_bucket),
            ('The sport of chess is recognized all over the world today', neut_bucket),
            ('Some celebrities drive high performance race cars', neut_bucket),
            ('All criminal charges will be taken seriously', neg_bucket),
            ('Antarctica is the coldest continent on earth', neut_bucket)]


In [3]:
writer = SummaryWriter('runs/sentimentAnalysis')
word2id = the_model.word2id
u_embeddings = the_model.u_embeddings
v_embeddings = the_model.v_embeddings

emo_strings = pos_bucket + neg_bucket + neut_bucket
#expand
emo_strings = [w for w in emo_strings if w in word2id]
emotions = torch.tensor([word2id[w] for w in emo_strings])
emotions = u_embeddings(emotions)

correct = 0
emo_strings_score = [[0, 0] for w in emo_strings]

for i, sen in enumerate(sentences):
    sen_arr = sen[0].split()
    sen_arr = torch.tensor([word2id[w] for w in sen_arr if w in word2id])
    sen_arr = v_embeddings(sen_arr)
    sen_arr = torch.sum(sen_arr, dim=0)

    #score = F.logsigmoid(torch.matmul(emotions, sen_arr.unsqueeze(1)))
    score = torch.cdist(emotions, sen_arr.unsqueeze(0)).squeeze()
    pred_emo = emo_strings[torch.argmin(score)] #changed

    print('sentence ', str(i+1), ': ', sen[0])
    print('Predicted emotion:', pred_emo)
    print('Actual emotion(s):', sen[1], '\n')

    if (pred_emo in sen[1]):
        correct += 1
        emo_strings_score[emo_strings.index(pred_emo)][0] += 1
    else:
        emo_strings_score[emo_strings.index(pred_emo)][1] += 1

correct /= len(sentences)
print('Accuracy:', correct*100, '%')

emo_strings = [ w + ' H ' + str(emo_strings_score[i][0]) + ' M ' + str(emo_strings_score[i][1]) for i,w in enumerate(emo_strings)]

writer.add_embedding(emotions, metadata=emo_strings)
writer.close()
%load_ext tensorboard

emo_strings

sentence  1 :  he lived a great life
Predicted emotion: happy
Actual emotion(s): ['excited', 'happy', 'strength', 'joy', 'inspiration'] 

sentence  2 :  I'm so sad
Predicted emotion: hate
Actual emotion(s): ['depressed', 'upset', 'hate', 'tired', 'warning', 'shock'] 

sentence  3 :  living life is a pleasure not known to many
Predicted emotion: happy
Actual emotion(s): ['excited', 'happy', 'strength', 'joy', 'inspiration'] 

sentence  4 :  I hate this
Predicted emotion: happy
Actual emotion(s): ['depressed', 'upset', 'hate', 'tired', 'warning', 'shock'] 

sentence  5 :  parents take care of children
Predicted emotion: happy
Actual emotion(s): ['neutral', 'formal', 'professional', 'passive', 'factual'] 

sentence  6 :  the interview was neither funny nor witty
Predicted emotion: joy
Actual emotion(s): ['depressed', 'upset', 'hate', 'tired', 'warning', 'shock'] 

sentence  7 :  you are such a liar
Predicted emotion: happy
Actual emotion(s): ['depressed', 'upset', 'hate', 'tired', 'warnin

['excited H 0 M 0',
 'happy H 11 M 19',
 'strength H 4 M 8',
 'joy H 0 M 1',
 'inspiration H 0 M 0',
 'depressed H 0 M 1',
 'upset H 0 M 0',
 'hate H 1 M 0',
 'tired H 0 M 0',
 'warning H 0 M 0',
 'shock H 0 M 0',
 'neutral H 1 M 0',
 'formal H 0 M 0',
 'professional H 0 M 0',
 'passive H 0 M 0']

In [7]:
%reload_ext tensorboard